In [150]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil

seed = 42

In [151]:
base_path = "data/state"
for directory in ["valid", "sample/valid", "sample/train"]:
    dir_path = os.path.join(base_path, directory)
    if not os.path.exists(dir_path): os.makedirs(dir_path)

In [152]:
df = pd.read_csv("data/state/driver_imgs_list.csv")

In [153]:
df.head(3)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg


In [154]:
subjects = df.subject.unique()
train_subjects = pd.Series(subjects).sample(frac=0.7, random_state=seed).values
valid_subjects = subjects[~np.isin(subjects, sample_train_subjects)]

In [157]:
valid_rows = df[df.subject.isin(valid_subjects)]
sample_train_rows = df[df.subject.isin(train_subjects)].sample(n=1000, random_state=seed)
sample_valid_rows = df[df.subject.isin(train_subjects) & \
                       (~df.index.isin(sample_train_rows.index))].sample(n=200, random_state=seed)

In [158]:
base_path = "data/state/train"
valid_path = "data/state/valid"
sample_train_path = "data/state/sample/train"
sample_valid_path = "data/state/sample/valid"

structure = [
    (valid_path, valid_rows),
    (sample_train_path, sample_train_rows),
    (sample_valid_path, sample_valid_rows)
]

for path, rows in structure:
    for _, (classname, img) in rows[['classname', 'img']].iterrows():
        origin = base_path + "/{}/{}".format(classname, img)
        destination_dir = path + "/{}".format(classname)
        os.makedirs(destination_dir, exist_ok=True)
        destination = destination_dir + "/{}".format(img)
        shutil.move(origin, destination)